# Motivation Experiments - Available Local DRAM on App/Mem Access-Level Latency

## Setup

Experiment notes: 
- 2-tier memory system - performance of VoltDB against available local DRAM
- CPU usage <900% (i.e., <9 cores)

Modify the `scalefactor` to be 5.

In [ ]:
# Replace function client() in Memtrade/apps/workload/voltdb/run.sh with this function

# function client() {
#     jars-ifneeded
#     java -classpath $APPNAME-client.jar:$APPNAME-procs.jar:$APPCLASSPATH com.MyTPCC \
#         --servers=localhost \
#         --transactions=580000 \
# 	--ratelimit=200000 \
#         --warehouses=256 \
#         --scalefactor=5 # NOTE: this is changed! Default is 22
# }

In [29]:
# /build_dir/qosmem/scripts/2tier-interference/start_volt_exp.sh diagnostics-all-local-cores
# At this point, results saved to /build_dir/voltdb/tests/test_apps/tpcc/xxx-$EXPNAME-$i.txt for i in 1..3

def print_voltdb_runs(expname='exp', with_memacc_lats=True):
    for i in range(1, 4):
        print(f"------------------------------------- Run {i} -------------------------------------")
        with open(f'/build_dir/qosmem/experiment-results/voltdb/tpcc/xxx-{expname}-{i}.txt', 'r') as file:
            lines = file.readlines()
            for line in lines[-22:]:
                print(line.strip())
        
        if with_memacc_lats:
            with open(f'/build_dir/qosmem/experiment-results/voltdb/mem-access-latency/monitor-{expname}-{i}.txt', 'r') as file:
                lines = file.readlines()
                # Get median latency from last 20 lines
                lats_ns = []
                for line in lines[-30:]:
                    lats_ns.append(float(line.split("latency = ")[-1].strip().split(" ns")[0]))
                lats_ns.sort()
                lat_ns = lats_ns[len(lats_ns)//2]
            print("Approximated Memory Access Latency:", lat_ns, "ns")
            print("=================================================================================")
       
        print()
        
def print_voltdb_runs_results(expname='exp', with_memacc_lats=True):
    run_summaries = []
    run_approx_memacc_lats = []
    for i in range(1, 4):
        with open(f'/build_dir/qosmem/experiment-results/voltdb/tpcc/xxx-{expname}-{i}.txt', 'r') as file:
            run_summary = file.readlines()[-22:]
            run_summaries.append(run_summary)
        
        if with_memacc_lats:
            with open(f'/build_dir/qosmem/experiment-results/voltdb/mem-access-latency/monitor-{expname}-{i}.txt', 'r') as file:
                lines = file.readlines()
                # Get median latency from last 20 lines
                lats_ns = []
                for line in lines[-30:]:
                    lats_ns.append(float(line.split("latency = ")[-1].strip().split(" ns")[0]))
                lats_ns.sort()
                approx_lat_ns = lats_ns[len(lats_ns)//2]
                run_approx_memacc_lats.append(approx_lat_ns)
    
    # Print in excel tab-separated format
    
    # Latency metrics:
    latency_metric_names = ["Average Latency", "Min Latency", "25-th Percentile Latency", "50-th Percentile Latency", "75-th Percentile Latency", "90-th Percentile Latency", "95-th Percentile Latency", "99-th Percentile Latency", "Max Latency"]
    latency_metrics_str = ""
    for j in range(len(latency_metric_names)):
        for i in range(3):
            latency_metrics_str += run_summaries[i][-10+j].split(" = ")[-1].split(" ms")[0] + "\t"
        latency_metrics_str += "\t" # for aggregated value in sheets
    
    # Throughput metric:
    throughput_metrics_str = ""
    for i in range(3):
        throughput_metrics_str += run_summaries[i][-18].split(" ")[-1].strip() + "\t"
    throughput_metrics_str += "\t" # for aggregated value in sheets
        
    approx_memacc_lat_str = ""
    if with_memacc_lats:
        # Approximated memory access latency metric:
        for i in range(3):
            approx_memacc_lat_str += f"{run_approx_memacc_lats[i]}" + "\t"
        
    print(latency_metrics_str + throughput_metrics_str + approx_memacc_lat_str)

## Diagnostic Experiments

### All 32 Local Cores, No Local DRAM Limit

In [17]:
print_voltdb_runs(expname="diagnostics-all-local-cores", with_memacc_lats=False)

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 179785 ms
Total transactions: 5800000
Transactions per second: 32260.76
Stock Level:     232560 total      1293.55 txn/s     77612.70 txn/m
Delivery:     231959 total      1290.20 txn/s     77412.13 txn/m
Order Status:     231856 total      1289.63 txn/s     77377.76 txn/m
Payment:    2493985 total     13872.04 txn/s    832322.50 txn/m
New Order:    2609640 total     14515.34 txn/s    870920.25 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 64.07 ms
- Min Latency = 3 ms
- 25-th Percentile Latency = 43 ms
- 50-th Percentile Latency = 48 ms
- 75-th Percentile Latency = 62 ms
- 90-th Percentile Latency = 115 ms

### 9 Local Cores, No Local DRAM Limit

In [18]:
print_voltdb_runs(expname="diagnostics-9-local-cores", with_memacc_lats=False)

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 179439 ms
Total transactions: 5800000
Transactions per second: 32322.96
Stock Level:     232560 total      1296.04 txn/s     77762.36 txn/m
Delivery:     231959 total      1292.69 txn/s     77561.40 txn/m
Order Status:     231856 total      1292.12 txn/s     77526.96 txn/m
Payment:    2493985 total     13898.79 txn/s    833927.44 txn/m
New Order:    2609640 total     14543.33 txn/s    872599.63 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 64.24 ms
- Min Latency = 0 ms
- 25-th Percentile Latency = 44 ms
- 50-th Percentile Latency = 49 ms
- 75-th Percentile Latency = 62 ms
- 90-th Percentile Latency = 108 ms

## Variable Local DRAM

In the following experiments,
- remote DRAM is not restricted (80GB)
- VoltDB is run on 9 local CPU cores, `1,3,5,7,9,11,13,15,17`, on CPU node 1

### 20GB local DRAM

In [19]:
print_voltdb_runs("local-dram-20gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 195242 ms
Total transactions: 5800000
Transactions per second: 29706.72
Stock Level:     232560 total      1191.14 txn/s     71468.23 txn/m
Delivery:     231959 total      1188.06 txn/s     71283.53 txn/m
Order Status:     231856 total      1187.53 txn/s     71251.88 txn/m
Payment:    2493985 total     12773.81 txn/s    766428.88 txn/m
New Order:    2609640 total     13366.18 txn/s    801970.88 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 70.02 ms
- Min Latency = 2 ms
- 25-th Percentile Latency = 46 ms
- 50-th Percentile Latency = 51 ms
- 75-th Percentile Latency = 66 ms
- 90-th Percentile Latency = 111 ms

### 18GB local DRAM

In [20]:
print_voltdb_runs("local-dram-18gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 192938 ms
Total transactions: 5800000
Transactions per second: 30061.47
Stock Level:     232560 total      1205.36 txn/s     72321.68 txn/m
Delivery:     231959 total      1202.25 txn/s     72134.78 txn/m
Order Status:     231856 total      1201.71 txn/s     72102.75 txn/m
Payment:    2493985 total     12926.35 txn/s    775581.31 txn/m
New Order:    2609640 total     13525.80 txn/s    811547.75 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 69.22 ms
- Min Latency = 2 ms
- 25-th Percentile Latency = 46 ms
- 50-th Percentile Latency = 50 ms
- 75-th Percentile Latency = 63 ms
- 90-th Percentile Latency = 118 ms

### 16GB local DRAM

In [21]:
print_voltdb_runs("local-dram-16gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 202434 ms
Total transactions: 5800000
Transactions per second: 28651.31
Stock Level:     232560 total      1148.82 txn/s     68929.13 txn/m
Delivery:     231959 total      1145.85 txn/s     68751.00 txn/m
Order Status:     231856 total      1145.34 txn/s     68720.47 txn/m
Payment:    2493985 total     12319.99 txn/s    739199.44 txn/m
New Order:    2609640 total     12891.31 txn/s    773478.75 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 72.31 ms
- Min Latency = 0 ms
- 25-th Percentile Latency = 47 ms
- 50-th Percentile Latency = 52 ms
- 75-th Percentile Latency = 66 ms
- 90-th Percentile Latency = 118 ms

### 12GB local DRAM

In [22]:
print_voltdb_runs("local-dram-12gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 205637 ms
Total transactions: 5800000
Transactions per second: 28205.04
Stock Level:     232560 total      1130.92 txn/s     67855.49 txn/m
Delivery:     231959 total      1128.00 txn/s     67680.14 txn/m
Order Status:     231856 total      1127.50 txn/s     67650.09 txn/m
Payment:    2493985 total     12128.09 txn/s    727685.75 txn/m
New Order:    2609640 total     12690.52 txn/s    761431.06 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 73.79 ms
- Min Latency = 3 ms
- 25-th Percentile Latency = 49 ms
- 50-th Percentile Latency = 54 ms
- 75-th Percentile Latency = 71 ms
- 90-th Percentile Latency = 113 ms

### 8GB local DRAM

In [23]:
print_voltdb_runs("local-dram-8gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 214588 ms
Total transactions: 5800000
Transactions per second: 27028.54
Stock Level:     232560 total      1083.75 txn/s     65025.07 txn/m
Delivery:     231959 total      1080.95 txn/s     64857.03 txn/m
Order Status:     231856 total      1080.47 txn/s     64828.23 txn/m
Payment:    2493985 total     11622.20 txn/s    697332.13 txn/m
New Order:    2609640 total     12161.17 txn/s    729669.88 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 77.06 ms
- Min Latency = 3 ms
- 25-th Percentile Latency = 51 ms
- 50-th Percentile Latency = 56 ms
- 75-th Percentile Latency = 69 ms
- 90-th Percentile Latency = 134 ms

### 4GB local DRAM

In [24]:
print_voltdb_runs("local-dram-4gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 218608 ms
Total transactions: 5800000
Transactions per second: 26531.51
Stock Level:     232560 total      1063.82 txn/s     63829.32 txn/m
Delivery:     231959 total      1061.07 txn/s     63664.37 txn/m
Order Status:     231856 total      1060.60 txn/s     63636.10 txn/m
Payment:    2493985 total     11408.48 txn/s    684508.81 txn/m
New Order:    2609640 total     11937.53 txn/s    716251.94 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 78.52 ms
- Min Latency = 6 ms
- 25-th Percentile Latency = 52 ms
- 50-th Percentile Latency = 57 ms
- 75-th Percentile Latency = 72 ms
- 90-th Percentile Latency = 131 ms

### 2GB local DRAM

In [25]:
print_voltdb_runs("local-dram-2gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 224977 ms
Total transactions: 5800000
Transactions per second: 25780.41
Stock Level:     232560 total      1033.71 txn/s     62022.34 txn/m
Delivery:     231959 total      1031.03 txn/s     61862.05 txn/m
Order Status:     231856 total      1030.58 txn/s     61834.59 txn/m
Payment:    2493985 total     11085.51 txn/s    665130.69 txn/m
New Order:    2609640 total     11599.59 txn/s    695975.13 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 80.86 ms
- Min Latency = 7 ms
- 25-th Percentile Latency = 53 ms
- 50-th Percentile Latency = 58 ms
- 75-th Percentile Latency = 75 ms
- 90-th Percentile Latency = 139 ms

### 1GB local DRAM

In [26]:
print_voltdb_runs("local-dram-1gb")

------------------------------------- Run 1 -------------------------------------
==================================== SUMMARY ====================================
----------------------------------- BANDWIDTH -----------------------------------
Time: 223382 ms
Total transactions: 5800000
Transactions per second: 25964.49
Stock Level:     232560 total      1041.09 txn/s     62465.19 txn/m
Delivery:     231959 total      1038.40 txn/s     62303.77 txn/m
Order Status:     231856 total      1037.94 txn/s     62276.10 txn/m
Payment:    2493985 total     11164.66 txn/s    669879.88 txn/m
New Order:    2609640 total     11682.41 txn/s    700944.56 txn/m
Reset Warehouse:          0 total         0.00 txn/s         0.00 txn/m
------------------------------------ LATENCY ------------------------------------
- Average Latency = 80.26 ms
- Min Latency = 6 ms
- 25-th Percentile Latency = 53 ms
- 50-th Percentile Latency = 58 ms
- 75-th Percentile Latency = 74 ms
- 90-th Percentile Latency = 135 ms

### Print to sheets

In [33]:
print("===== varying local dram ======")
local_dram_gb_arr = [1, 2, 4, 8, 12, 16, 18, 20]
for local_dram_gb in local_dram_gb_arr:
    print_voltdb_runs_results(f"local-dram-{local_dram_gb}gb")

print("===== diagnostics / baseline ======")
diagnostics_expnames = ["diagnostics-all-local-cores", "diagnostics-9-local-cores"]
for expname in diagnostics_expnames:
    print_voltdb_runs_results(expname=expname, with_memacc_lats=False)

===== varying local dram ======
80.26	80.14	79.90		6	7	6		53	52	52		58	57	57		74	74	74		135	135	133		155	156	154		244	240	232		4634	3947	3844		25964.49	26005.82	26085.35		677.891	671.817	669.868	
80.86	79.62	79.18		7	7	6		53	53	52		58	58	57		75	75	72		139	132	138		159	154	156		237	231	213		4404	3834	4111		25780.41	26171.16	26320.45		663.565	669.942	663.465	
78.52	77.55	77.70		6	1	3		52	51	52		57	57	57		72	72	72		131	127	127		151	147	150		233	219	220		4489	4612	3902		26531.51	26858.07	26822.67		634.684	630.306	630.074	
77.06	75.65	76.28		3	7	8		51	50	51		56	55	56		69	71	73		134	123	119		152	144	139		219	200	214		4014	3361	3362		27028.54	27526.64	27306.33		614.02	602.825	597.483	
73.79	74.32	73.40		3	0	4		49	49	49		54	54	54		71	67	70		113	125	115		133	144	134		222	199	206		4034	4054	4217		28205.04	28017.43	28347.30		551.24	554.664	553.434	
72.31	72.46	71.81		0	2	5		47	48	48		52	53	52		66	66	68		118	122	115		137	139	136		237	202	212		3565	4187	4209		28651.31	28715.57	28982.18		475.168	478